In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stroke"
cohort = "GSE47727"

# Input paths
in_trait_dir = "../../input/GEO/Stroke"
in_cohort_dir = "../../input/GEO/Stroke/GSE47727"

# Output paths
out_data_file = "../../output/preprocess/Stroke/GSE47727.csv"
out_gene_data_file = "../../output/preprocess/Stroke/gene_data/GSE47727.csv"
out_clinical_data_file = "../../output/preprocess/Stroke/clinical_data/GSE47727.csv"
json_path = "../../output/preprocess/Stroke/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Global peripheral blood gene expression study [HumanHT-12 V3.0]"
!Series_summary	"Samples were collected from 'control participants' of the Heart and Vascular Health (HVH) study that constitutes a group of population based case control studies of myocardial infarction (MI), stroke, venous thromboembolism (VTE), and atrial fibrillation (AF) conducted among 30-79 year old members of Group Health, a large integrated health care organization in Washington State."
!Series_overall_design	"Total RNA was isolated from peripheral collected using PAXgene tubes and gene expression was profiled using the Illumina platform."
Sample Characteristics Dictionary:
{0: ['age (yrs): 67', 'age (yrs): 54', 'age (yrs): 73', 'age (yrs): 52', 'age (yrs): 75', 'age (yrs): 59', 'age (yrs): 74', 'age (yrs): 76', 'age (yrs): 58', 'age (yrs): 60', 'age (yrs): 66', 'age (yrs): 70', 'age (yrs): 78', 'age (yrs): 77', 'age (yrs): 72', 'age (yrs): 57', 'age (yrs): 63', 'age (yrs): 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see this dataset contains gene expression data 
# profiled using the Illumina platform (HumanHT-12 V3.0)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary:

# For trait (Stroke): 
# This dataset doesn't explicitly mention stroke status in the sample characteristics.
# Looking at the background, these are "control participants" for various studies including stroke.
# So all participants seem to be controls (non-stroke), making this a constant feature.
trait_row = None  # No variable stroke status information is available

# For age: We can find age data in key 0
age_row = 0

# For gender: We can find gender data in key 1
gender_row = 1

# 2.2 Data Type Conversion
# Since trait_row is None, we still need to define convert_trait to follow the function signature
def convert_trait(value):
    return None  # Since trait data is not available

def convert_age(value):
    try:
        # Extract the age value after the colon and space
        if ':' in value:
            age_str = value.split(': ')[1]
            return float(age_str)  # Convert to float for continuous data
        return None
    except:
        return None

def convert_gender(value):
    try:
        # Extract the gender value after the colon and space
        if ':' in value:
            gender_str = value.split(': ')[1].lower()
            if 'female' in gender_str:
                return 0
            elif 'male' in gender_str:
                return 1
        return None
    except:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
# Save initial filtering result
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# According to the instructions, we should skip this step if trait_row is None
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
else:
    print("Clinical feature extraction skipped because trait data is not available.")


Clinical feature extraction skipped because trait data is not available.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Stroke/GSE47727/GSE47727_series_matrix.txt.gz


Gene data shape: (48803, 122)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers shown are ILMN_XXXXXX format, which are Illumina BeadArray probe IDs
# These are not human gene symbols but rather probe identifiers specific to Illumina microarray platforms
# They need to be mapped to proper gene symbols for downstream analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'nuID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILM

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the previous output we can see:
# - The gene expression data has identifiers like 'ILMN_1343291' in the index (probe IDs)
# - In the gene annotation data, the 'ID' column contains these same identifiers
# - The 'Symbol' column in the gene annotation data contains the gene symbols we want

prob_col = 'ID'   # Column containing probe identifiers
gene_col = 'Symbol'  # Column containing gene symbols

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First 5 rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping, shape: {gene_data.shape}")
print("First 10 gene symbols in the processed gene expression data:")
print(gene_data.index[:10])

# Save the gene expression data to a file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")

# Since we already determined that trait data is not available in this dataset (trait_row is None),
# we won't proceed with linking clinical and genetic data for this particular dataset.
print("Skipping linking clinical and genetic data because trait information is not available in this dataset.")


Gene mapping dataframe shape: (36157, 2)
First 5 rows of gene mapping:
             ID       Gene
0  ILMN_1725881   LOC23117
2  ILMN_1804174     FCGR2B
3  ILMN_1796063     TRIM44
4  ILMN_1811966  LOC653895
5  ILMN_1668162    DGAT2L3
Gene expression data after mapping, shape: (19120, 122)
First 10 gene symbols in the processed gene expression data:
Index(['A1BG', 'A1CF', 'A26A1', 'A26B1', 'A26C1B', 'A26C3', 'A2BP1', 'A2M',
       'A2ML1', 'A3GALT2'],
      dtype='object', name='Gene')


Gene expression data saved to: ../../output/preprocess/Stroke/gene_data/GSE47727.csv
Skipping linking clinical and genetic data because trait information is not available in this dataset.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# Use the gene_data variable that was already loaded in Step 4
print(f"Original gene data shape: {gene_data.shape}")

# Normalize gene symbols
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data_normalized.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. No clinical data available (trait_row is None from previous steps)
# We can't proceed with linking clinical and genetic data
print("No trait data available. Can't proceed with linking clinical and genetic data.")

# Create a minimal dataframe for validation purposes 
# (since we need a valid DataFrame when is_final=True)
empty_df = pd.DataFrame({"dummy": [0]})

# Since trait data is not available, the dataset is not usable for our purposes
# We pass is_biased=True to indicate unusable data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,
    is_biased=True,  # Setting is_biased to True since missing trait data makes dataset unusable
    df=empty_df,
    note="No stroke-specific trait data available in this dataset. Contains gene expression data from peripheral blood of older adults, but without clear stroke indicators."
)
print("Dataset deemed not usable due to missing trait data.")

Original gene data shape: (19120, 122)
Gene data shape after normalization: (18326, 122)


Normalized gene expression data saved to ../../output/preprocess/Stroke/gene_data/GSE47727.csv
No trait data available. Can't proceed with linking clinical and genetic data.
Abnormality detected in the cohort: GSE47727. Preprocessing failed.
Dataset deemed not usable due to missing trait data.
